# Intro to interactive plotting with Bokeh

As hydrologists we often work with time series data and geographical data. We measure a hydrological variable over time, often in multiple locations. After measuring for a certain time, some extreme events may occur, and we may be interested in looking at a specific period in time, or a geographical location. We want to interact with the data. More so, we want to look at the data interactively. In this notebook we will explore the [Bokeh python package](https://docs.bokeh.org/en/latest/index.html) for interactive plotting with Python. We will focus on geographical data and time series data, but many more kind of plots can be made with Bokeh. Interesting examples of these can be found in [the Bokeh Gallery](https://docs.bokeh.org/en/latest/docs/gallery.html).

### 1. Importing the necessary Python packages

Depending on the plot, we may need to import different `Models` from bokeh. Here we load a couple of models that we will explore in this notebook.

In [ ]:
import pandas as pd
import bokeh

from bokeh.plotting import figure, show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    ColorBar,
    DataTable,
    TableColumn,
)
from bokeh.layouts import row

# Ensure bokeh plots are displayed inline in the Jupyter notebook
bokeh.io.output_notebook()


### Load some example data

In this notebook we will explore time series from groundwater monitoring wells in Switzerland, collected from the [Swiss Federal Office for the Environment](https://www.bafu.admin.ch). The dataset contains 1) the time series in `heads.csv` and 2) the metadata for each well in `metadata.csv`. This is a convenient format to work with time series data and geographical data (X,Y locations plus some metadata). Note that if you have your own data, you can simply replace these files in the same format, and rerun this notebook to display your own data.

In [ ]:
## Import time series data
data = pd.read_csv("data/heads.csv", index_col=0, parse_dates=True)
data.head()

In [ ]:
# Import metadata
metadata = pd.read_csv("data/metadata.csv", index_col=0)
metadata.head()

### Pandas / Matplotlib plot

So, let's have a look at the time series data. We can do this by using the built-on Matplotlib plotting functions from Pandas as follows:

In [ ]:
data.plot(legend=False)

The above figure does not tell us a lot. We want to zoom in! We can do that by changing the y-limit or x-axis limits, but that involves quite a bit of programming. Moreover, perhaps we just want to have a quick look at the data? 

### Bokeh figure

Let's introduce Bokeh, an Python package for making interactive plots and webpages (not explored here, but possible). Making a Bokeh plot is a bit more complex, but once we are done the produced plot is very powerfull. Let's explore how to make an interactive version of the plot above.

In [ ]:
# Create a new Bokeh figure
p = figure(
    height=200,
    width=800,
    x_axis_type="datetime",  # This is important for time series
)

# We have to convert the DataFrame to a ColumnDataSource
source = ColumnDataSource(data)

# Plot the time series data
p.line(x="index", y="Davos", source=source)

# Show all time series
p.multi_line(
    xs=[data.index.values] * len(data.columns),
    ys=[data[name].values for name in data.columns],
)

# We have to show the plot
show(p)

### Plot a single time series

Let us consider plotting a single time series to explore some other features of Bokeh plotting. We will add some `tools` and dress up the plot.

In [ ]:
# Create a new Bokeh figure
p = figure(
    height=200,
    width=800,
    x_axis_type="datetime",
    x_axis_location="below",
    tools="pan,box_zoom,reset,save, wheel_zoom, hover",
    background_fill_color="#efefef",  # Or whatever color you want
)

# Plot the time series data
name = "Oberwichtrach"
p.line(x="index", y=name, source=source)

# Dress up the plot
p.title.text = name
p.xaxis.axis_label = "Date"
p.yaxis.axis_label = "Heads"

# We have to show the plot
show(p)


## Plot geographic data

Now we will plot the geographic data. We will use the `metadata` DataFrame to get the coordinates of each location. Bokeh uses the Mercator projection for plotting, while our data is provided in longitude and latitude. We use the Python package PyProj to transform the coordinates and add new columns to the metadata DataFrame. 


In [ ]:
# Convert latitude and longitude to x and y coordinates using the Mercator projection
from pyproj import Transformer

transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857")

metadata["x"], metadata["y"] = transformer.transform(metadata["lat"], metadata["lon"])

### Make a map

We are now ready to make a map. We will use the `figure` function from Bokeh to create a new plot.  We will set the `x_axis_type` and `y_axis_type` to "mercator" to use the Mercator projection and let Bokeh know that we are plotting geographical data. We will set the `tools` to "pan,box_zoom,reset,save, wheel_zoom, hover" to allow the user to interact with the plot. Finally, we will add a background map to inform us where we are.

In [ ]:
### Plot the geographic data with Bokeh

# Create a new Bokeh figure
p = figure(
    height=400,
    width=650,
    tools="pan, wheel_zoom, box_zoom, reset",
    background_fill_color="#ffffff",  # Or whatever color you want
    x_axis_type="mercator",
    y_axis_type="mercator",
)

# We have to convert the DataFrame to a ColumnDataSource
source = ColumnDataSource(metadata)

# Color the points according to one of the columns
col = metadata["Prec"].values
exp_cmap = LinearColorMapper(palette="Viridis256", low=min(col), high=max(col))

# Plot the geographic data
p.scatter(
    x="x",
    y="y",
    size=15,
    fill_alpha=0.8,
    source=source,
    fill_color={"field": "Prec", "transform": exp_cmap},
)

p.add_tile(
    "CartoDB Positron",
    retina=True,
)

# We have to show the plot
show(p)

### Adding a colorbar

Not bad, but what do these colors mean? We need to add a colorbar to the plot. We will do that, and re-draw the figure. This is a usefull option sometimes, as we will see later.

In [ ]:
# Add a color bar
bar = ColorBar(color_mapper=exp_cmap, location=(0, 0), title="Precipitation [mm/yr]")
p.add_layout(bar, "left")

show(p)

### Add a hovertool

Better. But what is the exact value of that single point? Or perhaps the value of another column in our metadata DataFrame? We can add such information by adding a `HoverTool` with specific user-provided description as follows: 

In [ ]:
# Add hovertool to p
p.add_tools(
    HoverTool(
        tooltips=[
            ("Name", "@index"),
            ("Precipitation", "@Prec mm/yr"),
            ("Evaporation", "@Evap mm/yr"),
            ("Temperature", "@Temp deg."),
        ]
    )
)

show(p)

### Interactive table and plot

Bokeh is really aimed at allowing you to interact with the data. In the example below we show how you can combine a Table with a geographical map to select and display certain parts of the data. This is just an example of the possibilities of Bokeh plotting, feel free to explore more!

In [ ]:
# Create a bokeh table with the metadata

columns = [
    TableColumn(field="index", title="Name"),
    TableColumn(field="Prec", title="Precipitation [mm/yr]"),
    TableColumn(field="Evap", title="Evaporation [mm/yr]"),
    TableColumn(field="Temp", title="Temperature [deg.]"),
]

table = DataTable(source=source, columns=columns, width=400, height=400)

show(row(p, table))